## Script for testing the expressions ${\mathbb P}(T_N<T_0), {\mathbb E}(T_{0,N}), {\mathbb E}(T_{0,N}|T_0<T_N)$

Given a discrete state space of length N with two absorbers, at positions $n=0,N$. In the LaTeX file we found an expression for:
* a walker's probability to pass position $n=N$ before passing $n=0$, depending on its starting position ($\Lambda(x)$);
* a walker's expected running time (time before it is absorbed), depending on its starting position ($\psi(x)$);
* a walker's expected running time, given that it ends up at $n=0$ before passing $n=N$ ($\tau(x)$).

With this script these expressions can be checked.

The first block does the initialisation. Fill in a number for p (probability of jumping to the right for every position), N (length of the state space) and M (number of walkers for Monte Carlo simulation.

The second block does a simulation (Monte Carlo) of M walkers walking through the state space.

The third block calculates the quantities by the formulae derived in the LaTeX file.

The fourth block displays the results in seven columns: the simulated and calculated/exact values for $\Lambda$, $\psi$ and $\tau$, depending on the starting position of the walker ($i$).

NOTE!! For p>q, the simulation of $\tau$ will take a very long time since position $n=N$ is much more likely to be reached than position $n=0$. There are three options:
* Don't use the 'Simulatie'-block for p>q, just calculate the exact values by running blocks 1,3,4;
* Disable the simulation of $\tau$ by commenting out the if statements in the while loop in the function Walk and by replacing "while pos!=0:" by "while pos not in (0,N):"
* In the while loop in the function Walk, replace "pos != 0" by "pos != N", "i==N" by "i==0" and "pos==N" by "pos==0". Take care that the resulting values for $\tau$ will be swapped, so $\tau(i=0)$ is actually $\tau(i=N)$.

In [184]:
#INITIALISATIE
p = .4         #Probability of jumping to the right
q = 1-p
N = 35         #Position of second absorber/length of state space
M = 10000      #Number of walkers per starting point

import random as random
import numpy as np

In [185]:
#SIMULATIE
def step(p,q,i):
    r = random.random()
    if r<p:
        i = i+1
    else:
        i = i-1
    return i

def walk(p,q,i,N):
    pos = i
    path = [pos]
    k = 0
    while pos != 0:   #not in (0,N):
        if i == N:
            break
        if pos == N:
            if k==0:
                k = 1
                path_Lambda_psi = path
            pos = i
            path = [pos]
            continue
            
        pos = step(p,q,pos)
        path.append(pos)
    if k==0:
        path_Lambda_psi = path
    return path_Lambda_psi, path

def M_walks(p,q,i,N,M):
    paths = []
    length= []
    length_tau= []
    end   = np.array(())
    for j in range(M):
        path, path_tau = walk(p,q,i,N)
        #paths.append(path)
        end = np.append(end, path[-1])
        leng = len(path)-1
        length.append(leng)
        leng_tau = len(path_tau)-1
        length_tau.append(leng_tau)
    return end, length, length_tau        #end: Gives a list of endpoints of the walkers, length: gives a list of lengths of paths of walkers

def res(p,q,N,M):
    Lambda = []              #List of probabilities to end in N for every starting position i
    psi = []                 #Average time to end in 0 or N for every starting position i
    tau = []                 #Average time to end in 0, for every starting position i
    for i in range (0,N+1):
        end, length, length_tau = M_walks(p,q,i,N,M)
        
        prob = sum(end)/N/M
        Lambda.append(prob)
        
        time = np.mean(length)
        psi.append(time)
        
#        cond = (end==0).astype(float)
#        print sum(cond)
#        clength = length*cond
        try:
            ctime = np.mean(length_tau) #sum(clength)/(sum(cond)) #np.mean(clength) #
            tau.append(ctime)
        except RuntimeWarning:
            tau.append("Infinite")
        
    return Lambda, psi, tau

LambdaS, psiS, tauS = res(p,q,N,M)

#print res(p,q,N,M)

In [186]:
#EXACTE UITDRUKKING
def phi(x,p,q):
    res = (q/p)**x
    return res

def phiD(Up,Low,p,q):
    res = (q/p)**(Up-Low)
    return res

def Lambda(y,N,p,q):
    num = 0
    den = 0
    for x in range(y):
        num = num + phi(x,p,q)
    for xi in range(N):
        den = den + phi(xi,p,q)
    res = num/den
    return res

def cP(N,p,q):
    sumT = 0
    sumN = 0
    for xi in range(0,N):
        sumT = sumT + fP(xi,p,q)
        sumN = sumN + phi(xi,p,q)
    res = sumT/sumN
    return res

def fP(x,p,q):
    sumA = 0
    for i in range(0,x):
        sumA = sumA + (1/p)*phiD(x,x-i,p,q) #(phi(x,p,q)/phi((x-i),p,q))
    #print 'f1 = '+str(sumA)
    return sumA

def psi(y,p,q,cst):
    sumB = 0
    sumC = 0
    for x in range(0,y):
    #    print 'x = '+str(x)
        sumB = sumB + phi(x,p,q)
        sumC = sumC + fP(x,p,q)
    #print 'psi1 = '+str(sumB)
    res = sumB*cst - sumC
    return res
 
def cT(N,p,q):
    sumT = 0
    sumN = 0
    for xi in range(0,N):
        sumT = sumT + fT(xi,N,p,q)
        sumN = sumN + phi(xi,p,q)
    res = sumT/sumN
    return res    

def fT(x,N,p,q):
    sumA = 0
    for i in range(0,x):
        sumA = sumA + phiD(x,x-i,p,q)*(1/p)*(1-Lambda(x-i,N,p,q))
    return sumA

def tau(y,N,p,q,cst):
    sumB = 0
    sumC = 0
    for x in range(0,y):
        sumB = sumB + phi(x,p,q)
        sumC = sumC + fT(x,N,p,q)
    try:
        res = (cst*sumB-sumC)/(1-Lambda(y,N,p,q))
    except ZeroDivisionError:
        res = "infinite"
    return res    


cP=cP(N,p,q)
cT=cT(N,p,q)

In [187]:
#OUTPUT
print 'q='+ str(q) + ', p=' + str(p) + ',cP=' +str(cP) + ',cT=' +str(cT)

row_format ="{:>3}" + "{:>17}" * 6
print row_format.format("i", "Lambda (sim)", "psi (sim)", "tau (sim)", "Lambda (exact)", "psi (exact)", "tau (exact)")
for i in range(N+1):
    print row_format.format(i, LambdaS[i], psiS[i], tauS[i], Lambda(i,N,p,q), psi(i,p,q,cP), tau(i,N,p,q,cT))

q=0.6, p=0.4,cP=4.99993990833,cT=4.9998884011
  i     Lambda (sim)        psi (sim)        tau (sim)   Lambda (exact)      psi (exact)      tau (exact)
  0              0.0              0.0              0.0              0.0              0.0              0.0
  1              0.0           4.8108           4.81083.43380979909e-07    4.99993990833    4.99989011797
  2              0.0           9.9884           9.98848.58452449774e-07    9.99984977082    9.99973044549
  3              0.0           14.728           14.7281.63105965457e-06    14.9997145646    14.9994986626
  4           0.0001          20.0977          20.08682.78997046176e-06    19.9995117552    19.9991625766
  5              0.0           24.707           24.7074.52833667256e-06    24.9992075411      24.99867583
  6              0.0          30.1922          30.19227.13588598874e-06      29.99875122    29.9979717826
  7              0.0          34.6908          34.6908 1.1047209963e-05    34.9980667383    34.9969548183


In [62]:
l  =np.array((4,2,4))
e=[]
print l[-2]
for el in l:
    e.append(el==1)
print e
#c[i] = (l[i]==1 for i in range(len(l)))
c = (l==4).astype(int)
print c*l

a = np.array((1,2,3))
b = 4
np.append(a,b)
print sum(a for a in range(2,4))

2
[False, False, False]
[4 0 4]
5


NameError: name 'NaN' is not defined

In [ ]:
#Alternative expression
def c2(N,p,q):
    sumT = 0
    sumN = 0
    for x in range(0,N):
        sumT = sumT + f2(x,p,q)
        sumN = sumN + phi(x,p,q)
    res = sumT/sumN
    return res

def f2(x,p,q):
    sumC = 0
    for i in range(0,x):
        sumC = sumC + phiD(x,x-i-1,p,q)+phiD(x,x-i,p,q) #(phi(x,p,q)/phi((x-i-1),p,q)) + (phi(x,p,q)/phi((x-i),p,q))
    return sumC

def psi2(y,p,q,cst):
    sumD = 0
    for x in range(0,y):
        sumD = sumD + phi(x,p,q)*cst - f2(x,p,q)
    return sumD

print
for n2 in range(0,N+1):
    print str(n2) + ' - ' + str(psi1(n2,p,q,c2))
    
    
#print "i - Lambda - psi - tau"
#for i in range (N+1):
#    print str(i) +" - "+ str(Lambda[i]) +" - "+ str(psi[i]) +" - "+ str(tau[i])
    

#print "i - Lambda - psi - tau"
#for i in range(0,N+1):
#    print str(i) +' - '+ str(Lambda(i,N,p,q)) +' - '+ str(psi(i,p,q,cP)) +' - '+ str(tau(i,p,q,cT))
    